<a href="https://colab.research.google.com/github/Coat0813/MySwiftApp/blob/main/finetuning_sentence_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [582]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [583]:
import os

In [584]:
DIR         = '/content/drive/MyDrive/colab/sentence_transformers/'
SOURCE_FILE = 'ldcc-20140209.tar.gz'
TARGET_FILE = 'livedoor/'

# データの用意

## データのダウンロード

### ライブドアニュースのデータのDL

データを読み込んだ後は、解凍させる データのDL自体はコマンドを使った方がラクなので、"wget"コマンドでDLする。

DLしたデータはtar.gz形式で圧縮されているので、それを解凍する必要があるので、"tarfile"ライブラリを使って、解凍している。

In [585]:
# tar.gzファイルの解凍のためのライブラリのインポート
import tarfile

In [586]:
# データのDL

!wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2025-01-20 21:40:10--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.20’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.58MB/s    in 8.1s    

2025-01-20 21:40:19 (1.04 MB/s) - ‘ldcc-20140209.tar.gz.20’ saved [8855190/8855190]



In [587]:
# ファイルの解凍

tar = tarfile.open(SOURCE_FILE, 'r:gz')
tar.extractall(TARGET_FILE)

In [588]:
# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "livedoor/text/") if os.path.isdir("livedoor/text/"+name)]
print(categories)

['README.txt', 'livedoor-homme', 'peachy', 'it-life-hack', 'sports-watch', 'movie-enter', 'kaden-channel', 'dokujo-tsushin', 'smax', 'CHANGES.txt', 'topic-news']
['livedoor-homme', 'peachy', 'it-life-hack', 'sports-watch', 'movie-enter', 'kaden-channel', 'dokujo-tsushin', 'smax', 'topic-news']


In [589]:
# ファイルの中身を確認してみる
file_name = "livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



フォルダ名称が「カテゴリ」で、フォルダの中に大量にテキストファイルが存在している。

テキストファイルは、
- 1行目：URL情報
- 2行目：タイムスタンプ（記事が公開された日時？）
- 3行目：タイトル
- 4行目：本文
というデータで入っている。

### テキストファイルをカテゴリ別に読み込む

In [590]:
import re
import pickle
import pandas as pd

In [591]:
# フォルダのファイルとディレクトリを確認
files     = os.listdir("livedoor/text/")
files_dir = [f for f in files if os.path.isdir(os.path.join("livedoor/text/", f))]
files_dir

['livedoor-homme',
 'peachy',
 'it-life-hack',
 'sports-watch',
 'movie-enter',
 'kaden-channel',
 'dokujo-tsushin',
 'smax',
 'topic-news']

In [592]:
list_df = []

for category in files_dir:
  path = "livedoor/text/" + category + "/"

  files      = os.listdir(path)
  files_file = [f for f in files if os.path.isfile(os.path.join(path, f)) and (re.match(category, f))]

  for file_name in files_file:
    file = path + file_name

    with open(file) as text_file:
      text = text_file.readlines()
      list_df.append([category, text[2], text[3]])
      #text[0] : # URL情報
      #text[1] : # タイムスタンプ
      #text[2] : # タイトル
      #text[3] : # 本文

with open('list_df.pkl', mode='wb') as f:
    pickle.dump(list_df, f)

In [593]:
list_df

[['livedoor-homme',
  '「仕事の後に飲みに行く風土、肝臓が悪くなってしまった…」 - 辛口説教部屋 vol.23\n',
  '「3年で転職は早すぎる？」「将来が見えない」\n'],
 ['livedoor-homme',
  '仕事と友情に効く、スコッチと名言\n',
  '日常生活の中でも大きなウェートを占める仕事——。生きがいとして頑張る人や、夢や希望を抱いて取り組む人、逆に生活の糧と割り切る人など、取り組み方も人それぞれといえる。しかし、どんな気持ちで仕事に取り組むにせよ、これを続けていくために必要となるのは、やはりモチベーションだろう。\n'],
 ['livedoor-homme',
  '闘魂に見る！部下のハートをがっちり掴む“オトコの生き方” 【新生活特集】\n',
  '「元気ですか〜\u3000元気があれば何でも出来る。元気があれば50周年も迎えられる！」\n'],
 ['livedoor-homme',
  '徹底検証！話題のセキュリティソフト「マカフィー オール アクセス 2012」をITニュース編集長が使い倒してみた\n',
  '最近では、毎日のようにセキュリティに関連したニュースが流れている。セキュリティというと、企業が対策を施すことのように思われがちだが、インターネットが家庭にまで普及している昨今、セキュリティ対策は個人でも必要欠くべからざるものとなっている。\n'],
 ['livedoor-homme',
  '年収1000万円のビジネスパーソンに聞いた「年収アップのために必要な資格・スキル」 - 年収1000万円研究所 vol.20\n',
  'こんにちは、「ビズリーチ年収1000万円研究所」所長の佐藤和男です。\n'],
 ['livedoor-homme',
  '「社風として夏休みが取りにくい…」 - 辛口説教部屋 vol.41\n',
  '「3年で転職は早すぎる？」「将来が見えない」\n'],
 ['livedoor-homme',
  '温泉に浸かるニホンザル、奇跡の列島「日本」の秘密に迫る\n',
  'この地球には、絶滅のおそれのある動物たちが、集中している場所がある。地表面積の3％にも満たない地域に、絶滅が危惧されている珍しい哺乳類、鳥類、両生類の実に75％が集まっている、その地域の呼称

In [594]:
df = pd.DataFrame(list_df, columns = ["category", "title", "text"])
df.head()

,category,title,text
0,livedoor-homme,「仕事の後に飲みに行く風土、肝臓が悪くなってしまった…」 - 辛口説教部屋 vol.23\n,「3年で転職は早すぎる？」「将来が見えない」\n
1,livedoor-homme,仕事と友情に効く、スコッチと名言\n,日常生活の中でも大きなウェートを占める仕事——。生きがいとして頑張る人や、夢や希望を抱いて取...
2,livedoor-homme,闘魂に見る！部下のハートをがっちり掴む“オトコの生き方” 【新生活特集】\n,「元気ですか〜 元気があれば何でも出来る。元気があれば50周年も迎えられる！」\n
3,livedoor-homme,徹底検証！話題のセキュリティソフト「マカフィー オール アクセス 2012」をITニュース編...,最近では、毎日のようにセキュリティに関連したニュースが流れている。セキュリティというと、企業...
4,livedoor-homme,年収1000万円のビジネスパーソンに聞いた「年収アップのために必要な資格・スキル」 - 年収...,こんにちは、「ビズリーチ年収1000万円研究所」所長の佐藤和男です。\n


In [595]:
df.groupby("category").count()["title"]

,title
category,
dokujo-tsushin,870
it-life-hack,870
kaden-channel,864
livedoor-homme,511
movie-enter,870
peachy,842
smax,870
sports-watch,900
topic-news,770


## Tripletデータの用意

- 学習とテスト用のデータに分割してファインチューニング前後の差を見れるようにするために、10,000ペアのデータを作る。
- つまり、anchor-positiveとanchor-negativeペアを10,000個作る

### anchor-positiveペアの作成

- ①7,367記事のインデックスで全組み合わせを作る。$7,367C_2 = 27,132,661$
- ②その組み合わせの中からランダムに1ペアを選ぶ
- ③選んだ組み合わせが同じカテゴリであれば、anchor-positiveとして登録する
- ④anchor-positiveとして登録された組み合わせは、組み合わせリストから削除する
- ①～④をを繰り返して、10,000ペア作り出す

In [596]:
import itertools
import random
import numpy as np

random.seed(0)

In [597]:
# ①7,367記事のインデックスで全組み合わせを作る。
# ペアの総数の定義
N_pair = 10_000

# 記事のレコード数を計算
N_RECORDS = df.shape[0]

list_records = [i for i in range(N_RECORDS)]

# インデックスの組み合わせを計算する
combinations = list(itertools.combinations(list_records, 2))

In [598]:
from collections import Counter

# カテゴリ数の確認
print(Counter(df['category']))

# 組み合わせをシャッフルして、全ペアを処理
random.shuffle(combinations)
combinations = combinations[:min(len(combinations), N_pair)]

list_anchor_text   = []
list_positive_text = []

list_anchor_cat   = []
list_positive_cat = []

for selected in combinations:
    anchor_candidate_ind   = selected[0]
    positive_candidate_ind = selected[1]

    anchor_candidate_cat   = df['category'][anchor_candidate_ind]
    positive_candidate_cat = df['category'][positive_candidate_ind]

    if anchor_candidate_cat == positive_candidate_cat:
        anchor_text   = df['text'][anchor_candidate_ind]
        positive_text = df['text'][positive_candidate_ind]

        list_anchor_text.append(anchor_text)
        list_positive_text.append(positive_text)
        list_anchor_cat.append(anchor_candidate_cat)
        list_positive_cat.append(positive_candidate_cat)

        if len(list_anchor_text) >= N_pair:
            break

print("Processing completed.")

Counter({'sports-watch': 900, 'it-life-hack': 870, 'movie-enter': 870, 'dokujo-tsushin': 870, 'smax': 870, 'kaden-channel': 864, 'peachy': 842, 'topic-news': 770, 'livedoor-homme': 511})
Processing completed.


In [599]:
# DataFrameで格納する。

dict_df_anc_pos = {
    "anchor"       : list_anchor_text,
    "positive"     : list_positive_text,
    "anchor_cat"   : list_anchor_cat,
    "positive_cat" : list_positive_cat
}

df_anc_pos = pd.DataFrame(dict_df_anc_pos)
df_anc_pos

,anchor,positive,anchor_cat,positive_cat
0,ファンがアイドルを応援するコンサートでは必需品のペンライト。古き良き80年代には、観客がささ...,９月といえば食欲の秋、そしてスポーツの秋。家電チャンネルでは、ウェイトコントロールを中心に楽...,kaden-channel,kaden-channel
1,エステやマッサージ、飲茶が魅力の台湾と、シーフードが絶品の南国の楽園シンガポール。どちらも...,2006年の発売以来、“女子力”のバイブルとして多くの女性に支持され、発行部数30万部を超...,peachy,peachy
2,ゼロから始めるスマートフォン \n,HTC J ISW13HTにソフトウェア更新！ \n,smax,smax
3,NHN Japanは、同社が提供する「livedoorニュース」で配信している記事の中から、...,テレビ事業の不振により、国内大手のソニー、シャープ、パナソニックの3社が過去最大ともいわれる...,kaden-channel,kaden-channel
4,OSとして圧倒的に普及しているWindowsだが、快適に動かすにはそれなりのハードウェアが必...,こんこん ふぉっくす紺子だよ♪\n,it-life-hack,it-life-hack
...,...,...,...,...
1119,電波が届かなくてネットに接続できる環境ではないようなケースにあって、それでも動画を再生したい...,ワコムは2012年3月1日、デジタル映像やデザイン制作向けに、高度な表現力と直感的な制作環境...,it-life-hack,it-life-hack
1120,5日、家庭用・業務用の計量器メーカーのタニタが「ヘルシー弁当」を発売すると、各メディアが報じ...,4日、Web版「週刊現代」が掲載した記事「民主も自民もついでに財界もこうしてぶっ潰す 橋下徹...,topic-news,topic-news
1121,Appleは11日（現地時間）、iPhoneやiPadなどのモバイル向けプラットフォーム「i...,shimajiro@mobiler \n,smax,smax
1122,AQUOS PHONE st SH-07Dにソフトウェア更新！ \n,REGZA Phone T-02Dが発表！ \n,smax,smax


### anchor-negativeペアの作成

- ①7,367記事の中から1つ記事を選ぶ
- ②anchor-positiveと別のカテゴリの場合は、anchor-negativeとして登録する

In [600]:
# データフレームのインデックスをリセット
df = df.reset_index(drop=True)
df_anc_pos = df_anc_pos.reset_index(drop=True)

count = 0

list_negative_text = []
list_negative_cat  = []

while count < N_pair:
    # ネガティブのインデックス候補を取得
    negative_candidate_ind = random.randint(0, len(df) - 1)

    # ネガティブカテゴリの候補を取得
    negative_candidate_cat = df['category'].iloc[negative_candidate_ind]

    # エラーチェック: countがdf_anc_posの範囲内か確認
    if count >= len(df_anc_pos):
        print("Index out of bounds for df_anc_pos.")
        break

    # anchorのカテゴリを取得
    positive_cat = df_anc_pos['anchor_cat'].iloc[count]

    # anchorカテゴリとネガティブカテゴリが違うかどうかを判定する
    if negative_candidate_cat != positive_cat:
        # anchorと異なるカテゴリの場合に登録
        negative_text = df['text'].iloc[negative_candidate_ind]
        list_negative_text.append(negative_text)
        list_negative_cat.append(negative_candidate_cat)

        # countを増やす
        count += 1

print(f"Negative samples created: {count}/{N_pair}")

Index out of bounds for df_anc_pos.
Negative samples created: 1124/10000


In [601]:
# DataFrameで格納する。

dict_df_neg = {
    "negative"     : list_negative_text,
    "negative_cat"   : list_negative_cat,
}

df_neg = pd.DataFrame(dict_df_neg)
df_neg

,negative,negative_cat
0,日本最大のポータルYahoo! JAPANを運営するヤフー株式会社（以下、Yahoo! JA...,it-life-hack
1,転職もせず引っ越しもせず、都内でのひとり暮らし歴12年になった久美さん(35歳)、実家の母か...,dokujo-tsushin
2,自然志向の高まりや、安全な食生活への関心から、見直されている農業。田んぼを借りての週末農業、...,dokujo-tsushin
3,3月22日、気象庁は東京・靖国神社のソメイヨシノが平年より6日早く開花したと発表しました。冬...,livedoor-homme
4,第22 回小説すばる新人賞を受賞した朝井リョウさんのベストセラー小説がついに映画化！ 突然部...,peachy
...,...,...
1119,倖田來未さんが、あの「ル パティシエ タカギ」のタカギシェフに、「食べてしまうのがもったい...,peachy
1120,「パソコンの知識がある」という言葉には二種類の意味があると筆者は思ってます。1つ目の意味は、...,it-life-hack
1121,リムジンの中で、優雅にシャンパンで乾杯…。そんな映画のようなシーンを「一度でいいから体験し...,peachy
1122,ブォーブォーッ。\n,dokujo-tsushin


### anchor-positive-negativeを結合する

In [602]:
df_data = pd.concat([df_anc_pos, df_neg], axis=1)
df_data = df_data[["anchor", "positive", "negative", "anchor_cat", "negative_cat"]]
df_data

,anchor,positive,negative,anchor_cat,negative_cat
0,ファンがアイドルを応援するコンサートでは必需品のペンライト。古き良き80年代には、観客がささ...,９月といえば食欲の秋、そしてスポーツの秋。家電チャンネルでは、ウェイトコントロールを中心に楽...,日本最大のポータルYahoo! JAPANを運営するヤフー株式会社（以下、Yahoo! JA...,kaden-channel,it-life-hack
1,エステやマッサージ、飲茶が魅力の台湾と、シーフードが絶品の南国の楽園シンガポール。どちらも...,2006年の発売以来、“女子力”のバイブルとして多くの女性に支持され、発行部数30万部を超...,転職もせず引っ越しもせず、都内でのひとり暮らし歴12年になった久美さん(35歳)、実家の母か...,peachy,dokujo-tsushin
2,ゼロから始めるスマートフォン \n,HTC J ISW13HTにソフトウェア更新！ \n,自然志向の高まりや、安全な食生活への関心から、見直されている農業。田んぼを借りての週末農業、...,smax,dokujo-tsushin
3,NHN Japanは、同社が提供する「livedoorニュース」で配信している記事の中から、...,テレビ事業の不振により、国内大手のソニー、シャープ、パナソニックの3社が過去最大ともいわれる...,3月22日、気象庁は東京・靖国神社のソメイヨシノが平年より6日早く開花したと発表しました。冬...,kaden-channel,livedoor-homme
4,OSとして圧倒的に普及しているWindowsだが、快適に動かすにはそれなりのハードウェアが必...,こんこん ふぉっくす紺子だよ♪\n,第22 回小説すばる新人賞を受賞した朝井リョウさんのベストセラー小説がついに映画化！ 突然部...,it-life-hack,peachy
...,...,...,...,...,...
1119,電波が届かなくてネットに接続できる環境ではないようなケースにあって、それでも動画を再生したい...,ワコムは2012年3月1日、デジタル映像やデザイン制作向けに、高度な表現力と直感的な制作環境...,倖田來未さんが、あの「ル パティシエ タカギ」のタカギシェフに、「食べてしまうのがもったい...,it-life-hack,peachy
1120,5日、家庭用・業務用の計量器メーカーのタニタが「ヘルシー弁当」を発売すると、各メディアが報じ...,4日、Web版「週刊現代」が掲載した記事「民主も自民もついでに財界もこうしてぶっ潰す 橋下徹...,「パソコンの知識がある」という言葉には二種類の意味があると筆者は思ってます。1つ目の意味は、...,topic-news,it-life-hack
1121,Appleは11日（現地時間）、iPhoneやiPadなどのモバイル向けプラットフォーム「i...,shimajiro@mobiler \n,リムジンの中で、優雅にシャンパンで乾杯…。そんな映画のようなシーンを「一度でいいから体験し...,smax,peachy
1122,AQUOS PHONE st SH-07Dにソフトウェア更新！ \n,REGZA Phone T-02Dが発表！ \n,ブォーブォーッ。\n,smax,dokujo-tsushin


### 作成したTripletデータを保存する

In [603]:
# 保存する
df_data.to_csv("/largest_data.csv", encoding='utf-8-sig', index=False)

# ファインチューニングする

## ライブラリのインストール

In [604]:
# sentence transformersを入れると、huggingfaceも自動で入るので、ラク
!pip install -U sentence-transformers

# 東北大のbertを使うには、fugachiが必要
!pip install fugashi

# 東北大のbertを使うには、unidic-liteが必要
!pip install fugashi[unidic-lite]

# matplotlibの日本語文字化け対策
!pip install japanize-matplotlib

In [605]:
#  ライブラリのインポート

# transformer系のライブラリ
import transformers
transformers.BertTokenizer = transformers.BertJapaneseTokenizer

# sentence transformersライブラリ
import sentence_transformers
from sentence_transformers            import SentenceTransformer
from sentence_transformers            import models
from sentence_transformers.losses     import TripletDistanceMetric, TripletLoss
from sentence_transformers.readers    import TripletReader
from sentence_transformers.evaluation import TripletEvaluator
# from sentence_transformers.datasets   import SentencesDataset # SentencesDataset を直接インポート
from sentence_transformers            import SentencesDataset, InputExample, losses # SentencesDatasetをsentence_transformersから直接インポート
from torch.utils.data                 import DataLoader
# ここでDatasetをsentence_transformersからインポートします
from sentence_transformers.datasets import Dataset # Datasetクラスをsentence_transformers.datasetsからインポート
# その他ライブラリ
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection  import train_test_split
import matplotlib.pyplot as plt
import japanize_matplotlib #日本語化matplotlib
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

# Datasetをここでインポートして、スコープ内にあることを確認します
# from datasets import Dataset # この行がエラーの原因だった可能性があります
# from sentence_transformers.datasets import Dataset # sentence_transformers.datasets から Dataset をインポートするように変更
# DatasetクラスはSentencesDatasetに含まれている可能性があります。
# 必要に応じて、以下のようにInputExampleを使用します。
# InputExample(texts=[row["anchor"], row["positive"], row["negative"]])

## モデルの定義


今回は、東北大がhugging faceで提供しているBERTモデルを使用する。

[モデルカード：hugging faceのページ](https://huggingface.co/cl-tohoku/bert-base-japanese-v3)

In [606]:
# モデル名を定義
# 今回は、東北大がhugging faceで提供しているBERTモデルを使用する
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'


# BERTモデルのロード
bert = models.Transformer(MODEL_NAME)


# プーリング層の定義
pooling = models.Pooling(
        bert.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True, # 埋め込みは mean で実施する
)


# BERTモデルとプーリング層を使って、Sentence BERTまたはSentence Transformersモデルを定義
model = SentenceTransformer(modules=[bert, pooling])

In [607]:
# 後で次の処理をしたいので、一旦上で定義したほぼ生の状態のsentence transformersモデルを保存しておく
# <処理>
# ファインチューニング前と後のモデルとで埋め込み精度の比較

#with open(DIR + 'model_before_fit.pickle', 'wb') as p:
#    pickle.dump(model, p)

import os
import pickle

# 保存ディレクトリが存在しない場合は作成
if not os.path.exists(DIR):
    os.makedirs(DIR, exist_ok=True)

# モデルを保存
with open(DIR + 'model_before_fit.pickle', 'wb') as p:
    pickle.dump(model, p)

print(f"Model saved at: {DIR + 'model_before_fit.pickle'}")

Model saved at: /content/drive/MyDrive/colab/sentence_transformers/model_before_fit.pickle


## 学習用とテスト用にデータを分割する

- 学習:8.5割
- テスト：1.5割

で分割する。

※ Google Colabの無料版だと、10,000ペアのデータを使うと、時間がかかりすぎてしまうので、今回は、10,000ペアのうち、2,000ペアを学習、テストで使うことにする。

---

Sentence transformersを使うときはデータを`InputExample`を使ってラッピングする。

In [608]:
# 学習用、検証用に分割する

df_ = df_data.head(5000)

df_train, df_eval = train_test_split(df_, test_size=0.15, random_state=42)

# 学習用データのセッティング
train_dataset = SentencesDataset([InputExample(texts=[row["anchor"], row["positive"], row["negative"]]) for index,row in df_train.iterrows()]
                                 ,model)

### ハイパーパラメータの調整

無料枠のGPUのメモリでは、バッチサイズを8以上にするとメモリ不足になってしまうので、↓の設定にすることとした。

In [609]:

# ハイパーパラメータの定義
# バッチサイズやウォームアップは Sentence BERT の論文の “3.1 Training Details” の記述に従いたいが、
# GPUのメモリを使い切ってしまう問題などが出たため、論文の設定よりもかなり小さくしている点に注意。

BATCH_SIZE   = 4
#NUM_EPOCH    = 10
NUM_EPOCH    = 2 #変更
EVAL_STEPS   = 100
WARMUP_STEPS = int(len(train_dataset) // BATCH_SIZE * 0.1)


### ファインチューニングの各種設定

evaluatorやDataloader、損失関数を定義する

In [610]:
evaluator = TripletEvaluator(df_eval.anchor.values.tolist()
                             ,df_eval.positive.values.tolist()
                             ,df_eval.negative.values.tolist()
                              )

# Dataloaderの定義
train_dataloader = DataLoader(
        train_dataset,
        shuffle    = False,
        batch_size = BATCH_SIZE,
)

# 損失関数の定義(ユークリッド距離で定義)
train_loss = TripletLoss(
        model           = model,
        distance_metric = TripletDistanceMetric.EUCLIDEAN,
        triplet_margin  = 1,
)

In [611]:
# モデルのトレーニングを行う
model.fit(
    train_objectives = [(train_dataloader, train_loss)],
    epochs           = NUM_EPOCH,
    evaluator        = evaluator,
    evaluation_steps = EVAL_STEPS,
    warmup_steps     = WARMUP_STEPS,
    output_path      = DIR + "/sbert",
    checkpoint_path  = DIR + "/check_points",
    checkpoint_save_steps = 100,
    checkpoint_save_total_limit = 999_999
)

NameError: name 'Dataset' is not defined

In [ ]:


# モデルのトレーニングを行う
model.fit(
    train_objectives = [(train_dataloader, train_loss)],
    epochs           = NUM_EPOCH,
    evaluator        = evaluator,
    evaluation_steps = EVAL_STEPS,
    warmup_steps     = WARMUP_STEPS,
    output_path      = DIR + "/sbert",
    checkpoint_path  = DIR + "/check_points",
    checkpoint_save_steps = 100,
    checkpoint_save_total_limit = 999_999
)

In [ ]:


# モデルのトレーニングを行う
model.fit(
    train_objectives = [(train_dataloader, train_loss)],
    epochs           = NUM_EPOCH,
    evaluator        = evaluator,
    evaluation_steps = EVAL_STEPS,
    warmup_steps     = WARMUP_STEPS,
    output_path      = DIR + "/sbert",
    checkpoint_path  = DIR + "/check_points",
    checkpoint_save_steps = 100,
    checkpoint_save_total_limit = 999_999
)

### 学習の実行

In [ ]:
# 学習済みモデルの保存

with open(DIR + 'model_after_fit.pickle', 'wb') as p:
    pickle.dump(model, p)

# 学習結果を評価する

ファインチューニング前後でモデルが学習をうまくできたかどうかを確認してみる

## ファインチューニング前後のモデルをロードする

In [ ]:
# ファインチューニング前後のモデルをロードする

# 前
with open(DIR + 'model_before_fit.pickle', 'rb') as p:
    model_before_fit = pickle.load(p)

# 後
with open(DIR + 'model_after_fit.pickle', 'rb') as p:
    model_after_fit = pickle.load(p)

## 評価する

- 2つの指標を使って評価する
- 『accuracy』
- 『diff』

### accuracy
positiveとnegativeとで類似度を比較したときに、類似度が高い方をpositiveと判定する2値分類のように考えると、positiveの方が類似度の高い時（=diff > 0の時）、positiveを正しくpositiveと判定していることになるので、その回数をカウントアップするような処理をしている。
いわゆる「accuracy」に近い考え方。

### diff
diffは大きくなればなるほど良い指標。

そもそもとして、理想は、posとの類似度が高く、negとの類似度が低いこと。

ここで定義している「diff」はposとの類似度が高いかつnegとの類似度が低い時にdiffがより大きくなるように定義している。
したがって、diffはで書ければでかいほど良いという評価になる。


In [ ]:
# 評価のための関数の定義
# accuracyとdiffを計算する
def evaluate(test_df, model):
  """
  評価指標を計算するための関数
  :param  : test_df : DataFrame. anchor(str) | positive(str) | negative(str) のカラム構成
  :model  : sentence transformer
  :return : dict. {"accuracy" : accuracy score, "diff", diff score}
  """
  correct  = 0
  avg_diff = 0

  # 各レコードのベクトル化と評価指標の計算を実行する
  for index, row in df_eval.iterrows():
    text_anchor   = row.anchor
    text_positive = row.positive
    text_negative = row.negative

    # ベクトル化(anchor - positive)
    vec_anchor   = model.encode(text_anchor)
    vec_positive = model.encode(text_positive)
    vec_negative = model.encode(text_negative)

    # cos類似度計算(anchor - positive、anchor - negative)
    score_pos = cosine_similarity([vec_anchor], [vec_positive])[0][0]
    score_neg = cosine_similarity([vec_anchor], [vec_negative])[0][0]

    # 類似度の差分の計算
    diff = score_pos - score_neg

    # 類似度差分の足し算
    avg_diff += diff

    # 類似度の高い文章ペアをちゃんと区別できている件数のカウント
    if diff > 0:
      correct += 1

  accuracy_score = correct  / len(test_df)
  diff_score     = avg_diff / len(test_df)

  return {"accuracy" : accuracy_score,
          "diff"     : diff_score}

In [ ]:
# ファインチューニング 前 のスコアを計算する

score_before = evaluate(test_df = df_eval
                        ,model  = model_before_fit)

In [ ]:
# ファインチューニング 後 のスコアを計算する

score_after = evaluate(test_df = df_eval
                       ,model  = model_after_fit)

## diffとaccuracyを可視化する

In [ ]:
data1  = [score_before["accuracy"], score_after["accuracy"]]
data2  = [score_before["diff"]    , score_after["diff"]]
labels = ['before', 'after']  # ラベル



f, ax1 = plt.subplots()
ax2    = ax1.twinx()


# 棒グラフの幅
bar_width = 0.35

# X軸の位置
r1 = range(len(data1))
r2 = [x + bar_width for x in r1]

# 棒グラフの描画
ax1.bar(r1, data1, color='blue', width=bar_width, edgecolor='black',   label='accuracy')
ax2.bar(r2, data2, color='orange', width=bar_width, edgecolor='black', label='diff')


# X軸の目盛りとラベル
ax1.set_xlabel('Data')

# Y軸の目盛りとラベル
ax1.set_ylabel("Accuracy")
ax2.set_ylabel("diff")

ax1.set_ylim([0.0, 1.0])
ax2.set_ylim([0.0, 0.5])



# 凡例の表示
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1.legend(h1+h2, l1+l2, bbox_to_anchor=(1.15, 1), loc='upper left', borderaxespad=0.3, fontsize=9)

# グラフのタイトル
plt.title('compare fine tuning results with raw model')

# グラフの表示
plt.show()


## t-SNEでファインチューニング前後でモデルが学習できているかどうかを確認する

In [ ]:
# textをベクトル化
def get_vector_and_tsne(df, model, perplexity = 5):
  """
  テキストデータをベクトル化して、それをt-SNEで2次元に圧縮するための関数
  :df    : DataFrame. text(str) カラムを持つDataFrame
  :model : sentence transformers
  :perplexity : perplexity. 一般的には、5～50に設定する
  :return : DataFrame. テキストをベクトル化し、x1 | x2 にそれぞれ格納したDataFrame

  """
  from sklearn.manifold import TSNE

  # t-SNEを定義
  tsne = TSNE(n_components=2, random_state = 0, perplexity = perplexity)

  # 2次元に変換
  X_embedded = tsne.fit_transform(model.encode(df.text.values.tolist()))

  # DataFrameに格納
  df_x_embedded = pd.DataFrame(X_embedded, columns = ["x1", "x2"])

  # オリジナルのDataFrameに結合
  df_result = pd.concat([df, df_x_embedded], axis=1)

  return df_result

In [ ]:
# categoryカラムとtextカラムだけにする
df_vis = df[["category", "text"]]

In [ ]:
# ファインチューニング 前

# t-SNEの実行
df_after_tsne = get_vector_and_tsne(df_vis, model_before_fit, perplexity = 50.0)

list_category = df_after_tsne.category.unique()

colors        =  ["r", "g", "b", "c", "m", "y", "k", "orange","pink"]

plt.figure(figsize = (10, 10))

for i , cat in enumerate(list_category):
    tmp_df = df_after_tsne[df_after_tsne.category == cat]

    plt.scatter(tmp_df['x1'],
                tmp_df['x2'],
                color = colors[i],
                alpha=0.5,
                label = cat)
    plt.legend()

In [ ]:
# ファインチューニング 後

# t-SNEの実行
df_after_tsne = get_vector_and_tsne(df_vis, model_after_fit, perplexity = 50.0)

list_category = df_after_tsne.category.unique()

colors        =  ["r", "g", "b", "c", "m", "y", "k", "orange","pink"]

plt.figure(figsize = (10, 10))

for i , cat in enumerate(list_category):
    tmp_df = df_after_tsne[df_after_tsne.category == cat]

    plt.scatter(tmp_df['x1'],
                tmp_df['x2'],
                color = colors[i],
                alpha=0.5,
                label = cat)
    plt.legend()